# Electronics Retail Company Dataset

Генерация датасета на тему Electronics Retail Company, для последующего анализа и построения модели машинного обучения.  

Структура создаваемого датасета:

1. **OrderID** – *int* – ID заказа, уникален, AUTO_INCREMENT
2. **Product** – *nvarchar(50)* – название электронного компонента, NOT NULL
3. **ProductCategory** – *nvarchar(50)* – категория электронного компонента, NOT NULL
4. **Brand** – *nvarchar(50)* – производитель электронного компонента, NULL
5. **Description** – *nvarchar(256)* – краткое описание электронного компонента, NULL
6. **StandardCost** – *money* – себестоимость электронного компонента, NOT NULL
7. **UnitPrice** – *money* – стоимость на продажу электронного компонента, NOT NULL
8. **DateRequest** – *datetime* – дата и время заказа, в пределах будних дней с 9:00 по 18:00 с 01.04.2024 по 31.03.2025, NULL
9. **QuantitySold** – *int* – количество проданных электронных компонентов за заказ, NOT NULL
10. **DiscountPrice** – *decimal(3, 2)* – скидка на электронный компонент, зависящая от количества на продажу, NOT NULL
11. **TotalPrice** – *money* – общая цена за заказ с учетом количества и скидки, NOT NULL
12. **CustomerName** – *nvarchar(50)* – ФИО покупателя, NOT NULL
13. **CustomerRegion** – *nvarchar(50)* – регионы Беларуси (всего 6 областей + столица Минск), NOT NULL
14. **CustomerCity** – *nvarchar(50)* – города принадлежащие областям, также + Минск, NOT NULL
15. **Canceled** – *bit* – отказ или возврат электронного компонента, NOT NULL
16. **Feedback** – *tinyint* – отзыв об электронном компоненте, оценка от 1 до 5, NULL

Датасет в формате *csv*. Количество строк - 5000. Все данные фиктивные.

In [1]:
# Импорт библиотек
import random
import string
import statistics
import csv
import datetime
import faker
import os

In [2]:
# -----------------------
# ПРОВЕРКА И УДАЛЕНИЕ ФАЙЛА ПЕРЕД СОЗДАНИЕМ
# -----------------------

# Имя файла
csv_file = r'D:\\Dataset\electronics_retail_dataset.csv'

# Проверяем, существует ли файл
if os.path.exists(csv_file):
    os.remove(csv_file)  # Удаляем файл
    print(f"Файл {csv_file} удалён.")

Файл D:\\Dataset\electronics_retail_dataset.csv удалён.


In [3]:
# -----------------------
# НАСТРОЙКА НАБОРОВ ПРОДУКТОВ
# -----------------------

# Устанавливаем фиксированный seed
SEED = 42
random.seed(SEED)

# Количество создаваемых продуктов
NUM_PRODUCTS = 100

# Список категорий
categories = [
    "Микросхема", "Оптопара", "Конденсатор", "Резистор", 
    "Индуктивность", "Предохранитель", "Диод", "Транзистор", "Резонатор",
    "Трансформатор", "Датчик", "Акустика", "Светодиод", "Дисплей",
    "Реле", "Разъем", "Переключатель", "Аккумулятор"
]

# Список наименований брендов
full_nouns = [
    "Arkham Devices", "Sumeru Instruments", "Digit Solutions", 
    "Quantum Systems", "Nexus Labs", "Fusion Technologies", "JPZ"
]

adjectives = ["Kepler", "Arion", "Delphini"]
nouns = ["Instruments", "Systems", "Technologies"]

# Список коротких слов
words = ["быстрый", "надёжный", "новый", "простой", "удобный", "точный", "сверхмощный", "легкий"]

# Список цен себестоимости
costs = [
    37.0, 2.65, 0.15, 0.14, 0.43, 0.36, 0.36, 0.60, 5.80, 
    26.0, 8.10, 1.10, 0.29, 18.0, 6.90, 5.60, 2.90, 20.50
]

# Вычисление среднего и стандартного отклонения себестоимости
stats = {"mean": statistics.mean(costs), "std": statistics.stdev(costs)}

# Минимальная и максимальная надбавка к себестоимости
MIN_MARKUP, MAX_MARKUP = 1.5, 2.0

# Функция для генерации парт-номеров с фиксированным форматом (9 символов)
def generate_part_number():
    part_number = ''.join(random.choices(string.ascii_uppercase + string.digits, k=9))
    return part_number

# Функция для генерации случайных категорий под парт-номера
def generate_category():
    return f"{random.choice(categories)}"

# Функция для генерации случайных названий брендов
def generate_brand():
    # Генерация полного названия бренда с 60% вероятности
    if random.random() > 0.4:
        return f"{random.choice(full_nouns)}"
    # Генерация объединенного названия бренда с 30% вероятности
    elif random.random() < 0.3:
        return f"{random.choice(adjectives)} {random.choice(nouns)}"
    # Бренд не указан (10%)
    else:
        return None

# Функция для генерации случайного описания (6 слов + 2 числа)
def generate_description():
    # Генерация описания с 90% вероятности
    if random.random() > 0.4:
        selected_words = random.sample(words, 6)  # Выбираем случайные 6 слов
        numbers = [str(random.randint(1, 999)), str(random.randint(1, 999))]  # Генерируем 2 числа
        elements = selected_words + numbers  # Объединяем слова и числа
        random.shuffle(elements)  # Перемешиваем порядок
        return " ".join(elements)
    # Описание не указано (10%)
    else:
        return None

# Функция для генерации себестоимости по нормальному распределению
def generate_cost():
    while True:
        value = round(random.gauss(stats["mean"], stats["std"]), 2)
        if value > 0:  # Проверяем, что значение положительное
            return value

# Функция для генерации цены на продажу на основе себестоимости
def generate_unit_prices(st_costs: iter):
    ut_prices = []
    for cost in st_costs:
        up_min, up_max = cost * MIN_MARKUP, cost * MAX_MARKUP
        ut_prices.append(round(random.uniform(up_min, up_max), 2))
    return ut_prices

In [4]:
# -----------------------
# ГЕНЕРАЦИЯ НАБОРОВ ПРОДУКТОВ
# -----------------------

# Пример генерации нескольких парт-номеров
PRODUCT_CODES = [generate_part_number() for _ in range(NUM_PRODUCTS)]

# Пример генерации нескольких категорий
PRODUCT_CATEGORIES = [generate_category() for _ in range(NUM_PRODUCTS)]

# Пример генерации нескольких брендов
BRANDS = [generate_brand() for _ in range(NUM_PRODUCTS)]

# Пример генерации нескольких описаний
DESCRIPTIONS = [generate_description() for _ in range(NUM_PRODUCTS)]

# Пример генерации нескольких цен себестоимости
STANDARD_COSTS = [generate_cost() for _ in range(NUM_PRODUCTS)]

# Пример генерации нескольких цен на продажу
UNIT_PRICES = generate_unit_prices(STANDARD_COSTS)

PRODUCTS_INFO = {
    "products": PRODUCT_CODES,
    "categories": PRODUCT_CATEGORIES,
    "brands": BRANDS,
    "descriptions": DESCRIPTIONS,
    "standard_costs": STANDARD_COSTS,
    "unit_prices": UNIT_PRICES
}

PRODUCTS_INFO

{'products': ['XAJI0Y6DP',
  'BHSAHXTHV',
  '3A3ZMF8MD',
  'D4V30T9NT',
  '3W5UZBIKC',
  'IDKWNNHJ7',
  'XVG0FN9XU',
  'Y41IBLJH7',
  '5LXO6QJIU',
  'JV6OH9SDB',
  'DW2PCN9T8',
  '4AZYTJXEP',
  'Q85JSG65K',
  'XVF1T2TAL',
  'A753LC58D',
  'RC11ERTJ5',
  'PHT0HL9XP',
  'SEIMVIHCW',
  'I64CIYHE7',
  'UR23GDPPQ',
  '0Y9DOM5IG',
  'QPKI7P5TB',
  '94874FRHO',
  'CN9J2QP89',
  'UZFK8UT0C',
  'VS4F8CGVY',
  'IE6IVWPVS',
  '7HZIOYKL1',
  'CQ99CHJ75',
  '5NF4ZW9XA',
  '3KX7EEDTJ',
  'VZHWJR64D',
  'PJA1WJ0TP',
  'AC56T4UFE',
  'L6246HID2',
  '5OWF75935',
  'A0L7253J2',
  'D54I3QK2I',
  'AGL58KXO9',
  'T7E8G8JDP',
  '0LVSNUJZA',
  '7TZ0YNCXL',
  'L4ZKLOOKE',
  'P7Y6WKTAK',
  'PUXQPHR62',
  'GDSWM31YI',
  'HAIR4COWG',
  'ZRIXA11DP',
  'G8SBI4Q2Y',
  '9V86WZS3T',
  '60RJIW1SW',
  'JCKJLTEIY',
  'ZCOTOHP6V',
  'Z41NAM148',
  'P0TVHHPBM',
  'YOFQEWAOU',
  'AXEQBNHL1',
  'N13JCAT9M',
  'X2X18HAFE',
  'YUHZ1GV0E',
  '38DALY8OZ',
  'CYWD14VE9',
  '2MPNSM43D',
  '8W3ZP08J3',
  'TRP0J43D5',
  'IQVNB4GH2'

In [5]:
# -----------------------
# НАСТРОЙКА И ГЕНЕРАЦИЯ НАБОРОВ ПОКУПАТЕЛЕЙ
# -----------------------

# Количество создаваемых покупателей
NUM_CUSTOMERS = 100

# Инициализация объекта Faker с русской локализацией
fake = faker.Faker(["ru_RU"])

# Генерация списка ФИО с помощью объекта fake
NAMES = [fake.last_name() + ' ' + fake.first_name() + ' ' + fake.middle_name() for _ in range(NUM_CUSTOMERS)]

# Список регионов РБ
regions = [
    "Брестская область", "Витебская область", 
    "Гомельская область", "Гродненская область", 
    "Могилёвская область", "Минская область",
    "Минск"
]

# Словарь - регион: список городов (может содержать разные наименования для того же города)
region_cities = {
    "Брестская область": ["Брест", "Барановичи", "Берёза", "Пинск"],
    "Витебская область": ["Витебск", "Полоцк", "Новополоцк", "Орша"],
    "Гомельская область": ["Гомель", "Мозырь", "Жлобин", "Речица"],
    "Гродненская область": ["Гродно", "Лида", "Слоним", "Волковыск"],
    "Могилёвская область": ["Могилёв", "Бобруйск", "Горки", "Осиповичи"],
    "Минская область": ["Молодечно", "Борисов", "Жодино", "Слуцк"],
    "Минск": ["Минск"]
}

# Функция для генерации региона/города
def generate_region_city():
    region = random.choice(regions)
    city = random.choice(region_cities[region])
    return region, city

# Генерация списка пар регион - город
CUSTOMER_WHEREABOUTS = [generate_region_city() for _ in range(NUM_CUSTOMERS)]

# Распаковываем список пар регион - город
CUSTOMER_REGIONS, CUSTOMER_CITIES = zip(*CUSTOMER_WHEREABOUTS)

CUSTOMERS_INFO = {
    "names": NAMES,
    "regions": CUSTOMER_REGIONS,
    "cities": CUSTOMER_CITIES
}

CUSTOMERS_INFO

{'names': ['Субботина Аркадий Игнатьевич',
  'Рябова Вероника Ивановна',
  'Орехов Амос Фокич',
  'Логинова Евстигней Гордеевич',
  'Шаров Виссарион Трифонович',
  'Пестова Евсей Даниловна',
  'Медведева Афиноген Венедиктович',
  'Константинова Станимир Захаровна',
  'Дмитриев Прокл Николаевна',
  'Ильина Лонгин Альбертовна',
  'Марков Ким Феоктистович',
  'Ковалев Лавр Харлампьевич',
  'Юдин Будимир Робертовна',
  'Киселев Модест Ярославович',
  'Гаврилов Остромир Дорофеевич',
  'Зуева Илья Демьянович',
  'Коновалов Наина Артемьевич',
  'Филиппов Натан Вениаминовна',
  'Нестеров Терентий Вячеславовна',
  'Шаров Болеслав Романовна',
  'Симонова Симон Макаровна',
  'Белозерова Вячеслав Филипповна',
  'Тетерина Парамон Федосьевич',
  'Белова Аполлон Адамович',
  'Горбачева Адам Евсеевич',
  'Архипова Мариан Абрамович',
  'Шаров Герасим Вадимовна',
  'Самсонова Онуфрий Тимофеевна',
  'Никитин Андрей Ефимовна',
  'Игнатьев Антип Болеславовна',
  'Трофимова Анна Федосеевич',
  'Колесникова 

In [6]:
# -----------------------
# НАСТРОЙКА И ГЕНЕРАЦИЯ ДАТЫ
# -----------------------

# Количество создаваемых дат
NUM_DATES = 5000

# Начальные и конечные метки дат
start_date = datetime.datetime(2024, 4, 1, 9, 0, 0)
end_date = datetime.datetime(2025, 3, 31, 18, 0, 0)

# Функция для генерации случайной даты (только будни, с 9 до 18)
def generate_random_datetime(start, end):
    while True:
        # Случайное число секунд между start и end
        total_seconds = int((end - start).total_seconds())
        rand_second = random.randint(0, total_seconds)
        dt = start + datetime.timedelta(seconds=rand_second)

        # Проверяем, что это будний день (Mon-Fri)
        if dt.weekday() < 5:  # 0-понедельник, 4-пятница
            # Проверяем время с 9:00 до 18:00
            if 9 <= dt.hour < 18:
                return dt.strftime("%Y-%m-%d %H:%M:%S")
            
DATE_REQUEST = [generate_random_datetime(start_date, end_date) for _ in range(NUM_DATES)]


# Сортируем список дат по возрастанию
DATE_REQUEST.sort()

DATE_REQUEST

['2024-04-01 09:19:58',
 '2024-04-01 09:47:51',
 '2024-04-01 09:51:38',
 '2024-04-01 10:34:54',
 '2024-04-01 10:43:05',
 '2024-04-01 10:49:24',
 '2024-04-01 11:18:03',
 '2024-04-01 11:40:39',
 '2024-04-01 12:17:45',
 '2024-04-01 13:36:53',
 '2024-04-01 13:59:42',
 '2024-04-01 14:03:37',
 '2024-04-01 14:58:52',
 '2024-04-01 15:18:00',
 '2024-04-01 15:22:14',
 '2024-04-01 15:42:33',
 '2024-04-01 15:43:33',
 '2024-04-01 16:51:33',
 '2024-04-01 16:51:36',
 '2024-04-01 17:05:13',
 '2024-04-02 09:53:36',
 '2024-04-02 10:31:12',
 '2024-04-02 11:42:50',
 '2024-04-02 13:40:06',
 '2024-04-02 13:45:04',
 '2024-04-02 13:57:29',
 '2024-04-02 14:09:55',
 '2024-04-02 14:48:17',
 '2024-04-02 15:03:12',
 '2024-04-02 15:49:36',
 '2024-04-02 16:06:23',
 '2024-04-02 17:55:44',
 '2024-04-03 09:02:33',
 '2024-04-03 09:05:16',
 '2024-04-03 09:35:32',
 '2024-04-03 09:48:20',
 '2024-04-03 10:18:44',
 '2024-04-03 10:37:23',
 '2024-04-03 10:58:04',
 '2024-04-03 11:58:44',
 '2024-04-03 12:55:57',
 '2024-04-03 13:

In [7]:
# -----------------------
# НАСТРОЙКА И ГЕНЕРАЦИЯ ОСНОВНОГО НАБОРА ЗАКАЗОВ
# -----------------------

# Количество создаваемых строк равно количеству дат 
NUM_ROWS = NUM_DATES

# Начальное и конечное количество в заказ
MIN_QTY, MAX_QTY = 1, 200

# Установка скидок в зависимости от количества в продажу
DISCOUNT_RANGE = {
            range(1, 10): 0.0,
            range(10, 50): 0.1,
            range(50, 100): 0.2,
            range(100, 201): 0.25
}

# Открываем файл CSV на запись
with open(csv_file, 'w', newline='', encoding='utf-8-sig') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    
    # Запишем заголовки
    writer.writerow([
        "OrderID", "Product", "ProductCategory", "Brand", "Description", "StandardCost", 
        "UnitPrice", "DateRequest", "QuantitySold", "DiscountPrice", "TotalPrice", 
        "CustomerName", "CustomerRegion", "CustomerCity", "Canceled", "Feedback"
    ])
    
    for order_id in range(1, NUM_ROWS + 1):
        # 1. OrderID
        order_id_val = order_id + 4838

        # 2. Product
        product_val = random.choice(PRODUCTS_INFO["products"])

        # Получение индекса со словаря по значению Product
        product_idx = next((i for i, val in enumerate(PRODUCTS_INFO["products"]) if val == product_val))

        # 3. ProductCategory
        cat_val = PRODUCTS_INFO["categories"][product_idx]

        # 4. Brand (иногда Unknown)
        brand_val = PRODUCTS_INFO["brands"][product_idx]
        if brand_val is None:
            brand_val = "Unknown"

        # 5. Description (иногда None)
        descr_val = PRODUCTS_INFO["descriptions"][product_idx]
        if descr_val is None:
            descr_val = ""

        # 6. StandardCost
        st_cost = PRODUCTS_INFO["standard_costs"][product_idx]

        # 7. UnitPrice
        unit_price = PRODUCTS_INFO["unit_prices"][product_idx]
        
        # 8. DateRequest
        date_req = DATE_REQUEST[order_id - 1]
        
        # 9. QuantitySold
        quantity_sold = random.randint(MIN_QTY, MAX_QTY)

        # 10. DiscountPrice
        discount_price = next((value for key, value in DISCOUNT_RANGE.items() if quantity_sold in key))

        # 11. TotalPrice
        total_price = round(quantity_sold * unit_price * (1 + discount_price), 2)
        
        # 12. CustomerName
        customer_name = random.choice(CUSTOMERS_INFO["names"])

        # Получение индекса со словаря по значению CustomerName
        customer_idx = next((i for i, val in enumerate(CUSTOMERS_INFO["names"]) if val == customer_name))

        # 13. CustomerRegion
        region_val = CUSTOMERS_INFO["regions"][customer_idx]

        # 14. CustomerCity
        city_val = CUSTOMERS_INFO["cities"][customer_idx]

        # 15. Canceled (бит) - примерно 10% отмены заказа
        canceled_val = 1 if random.random() < 0.1 else 0

        # Если отмена, то продажа 0
        total_price = 0 if canceled_val == 1 else total_price

        # 16. Feedback - 1..5 или 20% None
        if random.random() < 0.2:
            feedback_val = None
        else:
            feedback_val = random.randint(1, 5)
        
        # Запись строки в CSV
        writer.writerow([
            order_id_val,
            product_val,
            cat_val,
            brand_val,
            descr_val,
            st_cost,
            unit_price,
            date_req,
            quantity_sold,
            discount_price,
            total_price,
            customer_name,
            region_val,
            city_val,
            canceled_val,
            feedback_val
        ])

print("Файл electronics_retail_dataset.csv успешно сгенерирован!")

Файл electronics_retail_dataset.csv успешно сгенерирован!
